In [1]:
!pip install torch pandas numpy inflect transformers accelerate pillow tqdm scikit-learn bert-score rouge-score python-Levenshtein sentence-transformers


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.8 MB/s eta 0:00:00:00:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl 

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
import torch
import gc
import os
import pandas as pd
import re
import inflect
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from bert_score import score as bert_score
import evaluate
import warnings
import logging
from torch.cuda.amp import autocast
import time

# Suppress warnings
# warnings.filterWarnings("ignore")
logging.getLogger("transformers").setLevel(logging.ERROR)

# Set environment variable to avoid memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Setup inflect for number normalization
inflect_engine = inflect.engine()

# Normalize answer for single-word output
def normalize_answer(ans):
    ans = ans.strip().lower()
    ans = re.sub(r'[^\w\s]', '', ans)  # Remove punctuation
    if ans.isdigit():
        ans = inflect_engine.number_to_words(ans)
    words = ans.split()
    return words[0] if words else ""  # Return first word

# Parse BLIP-2 output to extract meaningful answer
def parse_blip2_output(raw_pred, prompt):
    # Remove the prompt
    answer_part = raw_pred.replace(prompt, "").strip()
    # Strip common prefixes
    prefixes = [
        "it's a", "it is a", "this is a", "is a", "it's", "it is", "this is"
    ]
    for prefix in prefixes:
        if answer_part.lower().startswith(prefix):
            answer_part = answer_part[len(prefix):].strip()
            break
    # Return the answer or a placeholder if empty
    return answer_part if answer_part else "NO_ANSWER"

# Model and processor
model_id = "Salesforce/blip2-opt-2.7b"
processor = Blip2Processor.from_pretrained(model_id, trust_remote_code=True)
processor.tokenizer.padding_side = 'left'  # Set left-padding for decoder-only model

# Load model with float16
try:
    model = Blip2ForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    ).to("cuda:0")
    model.eval()
except Exception as e:
    print(f"Failed to load {model_id}: {e}")
    exit()

# Load and validate dataset
val_path = "/kaggle/input/val1-dataset/val1.csv"
df = pd.read_csv(val_path)
df['answer'] = df['answer'].astype(str).str.lower()
df = df.dropna(subset=['question', 'answer'])
df["full_image_path"] = df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / Path(p.replace("\\", "/"))))

# Validate image paths
def validate_dataset(df):
    valid_rows = []
    for idx, row in df.iterrows():
        image_path = Path(row["full_image_path"])
        if image_path.exists():
            try:
                Image.open(image_path).convert("RGB")
                valid_rows.append(row)
            except:
                print(f"Invalid image at index {idx}: {row['full_image_path']}")
        else:
            print(f"Image not found at index {idx}: {row['full_image_path']}")
    return pd.DataFrame(valid_rows)
df = validate_dataset(df)
print(f"Valid samples: {len(df)}")

# VQA Dataset
from torch.utils.data import Dataset, DataLoader
class VQADataset(Dataset):
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["full_image_path"]).convert("RGB")
        image = image.resize((384, 384))  # BLIP-2 recommended resolution
        return {
            "image": image,
            "question": row["question"],
            "answer": row["answer"],
            "image_path": row["full_image_path"]
        }

# Custom collate function to handle PIL.Image.Image
def custom_collate_fn(batch):
    images = [item["image"] for item in batch]
    questions = [item["question"] for item in batch]
    answers = [item["answer"] for item in batch]
    image_paths = [item["image_path"] for item in batch]
    return {
        "image": images,
        "question": questions,
        "answer": answers,
        "image_path": image_paths
    }

vqa_dataset = VQADataset(df)
dataloader = DataLoader(vqa_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

# Process samples
correct = 0
total = 0
all_preds = []
all_labels = []

for batch_idx, batch in enumerate(tqdm(dataloader, desc="Processing batches", ncols=100)):
    images = batch["image"]
    questions = batch["question"]
    labels = batch["answer"]
    image_paths = batch["image_path"]
    
    batch_start_time = time.time()
    valid_samples = 0
    
    try:
        # Format questions with simpler prompt
        formatted_questions = [f"Question: {q} Answer:" for q in questions]
        inputs = processor(images=images, text=formatted_questions, return_tensors="pt", padding=True)
        inputs = {k: v.to("cuda:0", dtype=torch.float16 if v.dtype in [torch.float, torch.float32] else v.dtype) for k, v in inputs.items()}
        
        with torch.no_grad(), autocast():
            generated_ids = model.generate(**inputs, max_new_tokens=5, do_sample=False, num_beams=3)
        outputs = processor.batch_decode(generated_ids, skip_special_tokens=True)
        
        # Debug raw token IDs for first 3 batches
        if batch_idx < 3:
            print(f"Batch {batch_idx + 1} Token IDs: {generated_ids.tolist()}")
        
        for pred_raw, label, question, prompt, img_path in zip(outputs, labels, questions, formatted_questions, image_paths):
            # Parse the output to extract the answer
            parsed_answer = parse_blip2_output(pred_raw, prompt)
            pred = normalize_answer(parsed_answer)
            label = normalize_answer(label)
            if pred and pred != "no_answer":  # Skip placeholder
                all_preds.append(pred)
                all_labels.append(label)
                if pred == label:
                    correct += 1
                total += 1
                valid_samples += 1
                if total <= 10:
                    print(f"Sample {total}: Question: '{question}', Pred: '{pred}', Label: '{label}', Raw Pred: '{pred_raw}', Parsed Answer: '{parsed_answer}', Image: '{img_path}'")
            else:
                print(f"Skipped sample in batch {batch_idx + 1}: Invalid pred ('{pred}') for question: '{question}', Raw Pred: '{pred_raw}', Parsed Answer: '{parsed_answer}'")
    
    except Exception as e:
        print(f"Error in batch {batch_idx + 1}: {e}")
        continue
    
    # Log batch summary
    if valid_samples < len(questions):
        print(f"Batch {batch_idx + 1}: Processed {valid_samples}/{len(questions)} valid samples")
    
    if (total // 100) > ((total - valid_samples) // 100):
        torch.cuda.empty_cache()
        gc.collect()
        print(f"Processed {total} samples, GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB, Batch Time: {time.time() - batch_start_time:.2f}s")

# Compute exact-match accuracy
accuracy = correct / total if total > 0 else 0
print(f"\nBLIP-2 Exact-match Accuracy: {accuracy:.3f} ({correct}/{total})")

# Compute BERTScore
if all_preds and all_labels:
    P, R, F1 = bert_score(all_preds, all_labels, lang="en", model_type="roberta-large")
    print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

# Compute METEOR and ROUGE
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")
meteor_score = meteor.compute(predictions=all_preds, references=all_labels)
rouge_score = rouge.compute(predictions=all_preds, references=all_labels)
print(f"METEOR: {meteor_score['meteor']:.4f}")
print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

BERTScore - Precision: 0.9770, Recall: 0.9784, F1: 0.9773


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


METEOR: 0.1051
ROUGE-L: 0.1858
